In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
data_dir = r'F:\weightbodysize\data\train'
checkpoint_path = 'F:/weightbodysize/addblDenseNet201.ckpt'

In [ ]:
data_root = pathlib.Path(data_dir)
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
all_image_labels = [pathlib.Path(path).parent.name for path in all_image_paths]


In [ ]:
weight_labels = [[label.split('_')[1]] for label in all_image_labels]
new_weight_labels=[]
for n in weight_labels:
    new_weight_labels.append(float(n[0]));  
weight_labels=new_weight_labels

bw_labels = [[label.split('_')[2]] for label in all_image_labels]
new_bw_labels=[]
for n in bw_labels:
    new_bw_labels.append(float(n[0]));    
bw_labels=new_bw_labels

bh_labels = [[label.split('_')[3]] for label in all_image_labels]
new_bh_labels=[]
for n in bh_labels:
    new_bh_labels.append(float(n[0]));    
bh_labels=new_bh_labels

hw_labels = [[label.split('_')[4]] for label in all_image_labels]
new_hw_labels=[]
for n in hw_labels:
    new_hw_labels.append(float(n[0]));    
hw_labels=new_hw_labels

hh_labels = [[label.split('_')[5]] for label in all_image_labels]
new_hh_labels=[]
for n in hh_labels:
    new_hh_labels.append(float(n[0]));    
hh_labels=new_hh_labels

bl_labels = [[label.split('_')[6]] for label in all_image_labels]
new_bl_labels=[]
for n in bl_labels:
    new_bl_labels.append(float(n[0]));    
bl_labels=new_bl_labels

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices((weight_labels, bw_labels, bh_labels, hw_labels, hh_labels, bl_labels))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
image_count = len(all_image_paths)

test_count = int(image_count*0.3)
train_count = image_count - test_count
train_data = image_label_ds.skip(test_count)
test_data = image_label_ds.take(test_count)

BATCH_SIZE = 16
train_data = train_data.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

test_data = test_data.batch(BATCH_SIZE)

In [ ]:
DenseNet201 = keras.applications.densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
DenseNet201.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = DenseNet201(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)
out_bw = tf.keras.layers.Dense(1,name='out_bw')(x)
out_bh = tf.keras.layers.Dense(1,name='out_bh')(x)
out_hw = tf.keras.layers.Dense(1,name='out_hw')(x)
out_hh = tf.keras.layers.Dense(1,name='out_hh')(x)
out_bl = tf.keras.layers.Dense(1,name='out_bl')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight, out_bw, out_bh, out_hw, out_hh, out_bl])
model.summary()

In [ ]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(),
              loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=150,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('DenseNet201_loss.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('DenseNet201_val_loss.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

